In [1]:
import pandas as pd
import pyarrow as pa
import datetime as dt
import os
import duckdb

from deltalake import DeltaTable, write_deltalake
from dotenv import load_dotenv
from pathlib import Path

In [2]:
conn = duckdb.connect()

In [3]:
bucket_name = "forums-analyses-bucket"
object_name = "raw_youtube_videos_comments"
folder_name = ""

In [4]:
DELTA_VIDEOS_COMMENTS_PATH = os.path.join("s3://", bucket_name, folder_name, object_name).replace("\\", "/")

In [5]:
# load env variables
env_dir = Path('../../').resolve()
load_dotenv(os.path.join(env_dir, '.env'))

# get env variables
aws_creds = {
    "access_key": os.environ.get("AWS_ACCESS_KEY_ID"),
    "secret_key": os.environ.get("AWS_SECRET_ACCESS_KEY"),
    "region": os.environ.get("AWS_REGION_NAME"),
}

# inspect potential duplicates that may have been scraped in the data extractor scripts
[0, comment, fjOeJssZX_Q, UgxHZSCawcylfpfa5dZ4AaABAg, UC4sfyOmp6fQ1TOo_POiRYQQ, UCnLuLSV-Oi0ctqjxGgxFlmg, None, They are good friends, They are good friends, 2025-12-24T07:15:18Z, 2025-12-24T07:15:18Z, 0.0, @chritianjohn9701, http://www.youtube.com/@chritianjohn9701, 2025-12-25 02:04:33.671992]
[1, comment, fjOeJssZX_Q, UgxHZSCawcylfpfa5dZ4AaABAg, UC4sfyOmp6fQ1TOo_POiRYQQ, UCnLuLSV-Oi0ctqjxGgxFlmg, None, They are good friends, They are good friends, 2025-12-24T07:15:18Z, 2025-12-24T07:15:18Z, 0.0, @chritianjohn9701, http://www.youtube.com/@chritianjohn9701, 2025-12-25 02:04:33.671992]
[2, comment, fjOeJssZX_Q, UgxHZSCawcylfpfa5dZ4AaABAg, UC4sfyOmp6fQ1TOo_POiRYQQ, UCnLuLSV-Oi0ctqjxGgxFlmg, None, They are good friends, They are good friends, 2025-12-24T07:15:18Z, 2025-12-24T07:15:18Z, 0.0, @chritianjohn9701, http://www.youtube.com/@chritianjohn9701, 2025-12-25 02:04:33.671992]

may have all be potentially scraped as duplicate records and if records like them don't exist in the delta table then delta inserts all these as distinct records even if we define a predicate of comment_id being the key that dleta must use to check for existing records to make necessary updates or inserts.

In [ ]:
delta_videos_comments_table = DeltaTable(DELTA_VIDEOS_COMMENTS_PATH, storage_options=aws_creds)

In [7]:
videos_comments_pa_table = delta_videos_comments_table.to_pyarrow_table()

In [8]:
conn.sql("""
    SELECT * FROM videos_comments_pa_table
""")

┌─────────┬─────────────┬────────────────────────────┬──────────────────────────┬───────────────────────────────────┬───────────────────┬────────────────────────────┬────────────────────────────┬──────────────────────┬──────────────────────┬────────────┬─────────────────────┬──────────────────────────────────────────┬────────────────────────────┐
│  level  │  video_id   │         comment_id         │    author_channel_id     │ channel_id_where_comment_was_made │ parent_comment_id │       text_original        │        text_display        │     published_at     │      updated_at      │ like_count │ author_display_name │            author_channel_url            │          added_at          │
│ varchar │   varchar   │          varchar           │         varchar          │              varchar              │      varchar      │          varchar           │          varchar           │       varchar        │       varchar        │   int64    │       varchar       │                 varchar  

So if I had say ff. records:
```
| level | videoid | comment_id | added_at |
reply, somevideoid1, somecommentid1, 2025-01-01 00:00:00.300
reply, somevideoid1, somecommentid1, 2025-01-01 00:00:00.300
reply, somevideoid1, somecommentid1, 2025-01-01 00:00:00.500
reply, somevideoid1, somecommentid1, 2025-01-01 00:00:00.750
reply, somevideoid1, somecommentid1, 2025-01-01 00:00:00.750
```
I would want to write a query that would delete the records with 2025-01-01 00:00:00.300, 2025-01-01 00:00:00.500, and 2025-01-01 00:00:00.750 as their added_at timestamp while retaining only one of the most recent record which in this case is 2025-01-01 00:00:00.750

There are about 3 ways to approach deleting these duplicate rows with these certain timestamps
1. using a ROW_NUMBER() to be able to retain the record with the maximum added_at timestamp as this is the most recent and will do so regardless if it has duplicates
```
| level | videoid | comment_id | added_at | most_recent_rank
reply, somevideoid1, somecommentid1, 2025-01-01 00:00:00.300, 5
reply, somevideoid1, somecommentid1, 2025-01-01 00:00:00.300, 4
reply, somevideoid1, somecommentid1, 2025-01-01 00:00:00.500, 3
reply, somevideoid1, somecommentid1, 2025-01-01 00:00:00.750, 2
reply, somevideoid1, somecommentid1, 2025-01-01 00:00:00.750, 1
```
here we want to filter and retain the record with the most_recent_rank of 1 since it is the most recent and we want to keep it in our table. This moreover will not only retain one record of the duplicate record but all records that do not have a duplicate record since they are the only ones and will have a most_recent_rank of 1

once done we can execute an insert overwrite transaction of the whole table.
```
-- 2. Overwrite your original table with the clean data
INSERT OVERWRITE your_delta_table_name
SELECT * FROM clean_comments;
```

2. but the above is much more costly and overwriting 4m rows or more is not a good idea. So another is through a 2 way transaction. First is where we select the duplicate records
```

# identifying comment_ids with duplicates

In [9]:
conn.sql("""
    CREATE OR REPLACE TEMPORARY TABLE duplicate_comments AS (
        SELECT
            comment_id,
            COUNT(comment_id) AS n_duplicates
        FROM videos_comments_pa_table
        GROUP BY comment_id
        HAVING COUNT(comment_id) > 1
        ORDER BY n_duplicates DESC
    );
""")

In [10]:
conn.sql("SELECT * FROM duplicate_comments")

┌────────────────────────────┬──────────────┐
│         comment_id         │ n_duplicates │
│          varchar           │    int64     │
├────────────────────────────┼──────────────┤
│ UgzUwGsxgcpfwQcdVrJ4AaABAg │        12500 │
│ Ugw0mWXyiCv_G4K1ch54AaABAg │        12500 │
│ UgxBz7EV3q2lYeNtbKt4AaABAg │        12500 │
│ UgyQu70_1xaIcwgIuAF4AaABAg │        12500 │
│ UgwCEQ5Acjz78Neh9UR4AaABAg │        12500 │
│ Ugy7pv-I9DEMGfh3uYV4AaABAg │        12500 │
│ UgzIg8wfwHx_AF93omN4AaABAg │        12500 │
│ Ugx_dWL334dp0YZ_8hd4AaABAg │        12500 │
│ UgzZA4w7HiC5f4Qh7bR4AaABAg │        12500 │
│ UgwdaNFtxB86CGyCQJ54AaABAg │        12500 │
│             ·              │            · │
│             ·              │            · │
│             ·              │            · │
│ Ugyfc-e3p5tIQxKuSWB4AaABAg │            3 │
│ UgxnMmIv3qOS8Q0VrqF4AaABAg │            3 │
│ UgwzrRAMItkMmGlmyyp4AaABAg │            3 │
│ UgyBZU7lbPZCOpNG9ol4AaABAg │            3 │
│ Ugx0Zyi-v1qgblWhFRl4AaABAg │    

# extracting strictly only one among records with duplicates that are the most recent

In [19]:
conn.sql("""
    CREATE OR REPLACE TEMPORARY TABLE ranked_comments AS (
        SELECT
            *,
            ROW_NUMBER() OVER(
                PARTITION BY comment_id
                ORDER BY added_at DESC
            ) AS most_recent_rank
        FROM videos_comments_pa_table
        WHERE comment_id IN (SELECT comment_id FROM duplicate_comments)
    );
""")

In [20]:
conn.sql("""
    SELECT 
        * 
    FROM ranked_comments 
    WHERE comment_id LIKE 'UgyzlVOY0j9Uy3FrO4l4AaABAg'
    ORDER BY most_recent_rank ASC;
""")

┌─────────┬─────────────┬────────────────────────────┬──────────────────────────┬───────────────────────────────────┬───────────────────┬───────────────┬──────────────┬──────────────────────┬──────────────────────┬────────────┬─────────────────────┬────────────────────────────────────┬────────────────────────────┬──────────────────┐
│  level  │  video_id   │         comment_id         │    author_channel_id     │ channel_id_where_comment_was_made │ parent_comment_id │ text_original │ text_display │     published_at     │      updated_at      │ like_count │ author_display_name │         author_channel_url         │          added_at          │ most_recent_rank │
│ varchar │   varchar   │          varchar           │         varchar          │              varchar              │      varchar      │    varchar    │   varchar    │       varchar        │       varchar        │   int64    │       varchar       │              varchar               │         timestamp          │      int64    

In [21]:
conn.sql("""
    CREATE OR REPLACE TEMPORARY TABLE most_recent_dups AS (
        SELECT
            * EXCLUDE(most_recent_rank)
        FROM ranked_comments
        WHERE most_recent_rank = 1
    );
""")

In [22]:
conn.sql("SELECT * FROM most_recent_dups")

┌─────────┬─────────────┬───────────────────────────────────────────────────┬──────────────────────────┬───────────────────────────────────┬────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────┬──────────────────────┬────────────┬────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────────────┐
│  level  │  video_id   │                    comment_id                     │    author_channel_id     │ channel_id_where_comment_was_made │     parent_comment_id      │                                         text_original                                          │                                          text_display                                          │     published_at     │      updated_at      │ like_count │  auth

# because we have now extracted both the set of duplicate records and these duplicate records most recent record versions, we will now commence with the first stage transaction: the deletion of the duplicate records first.
And rather than an overwrite operation we instead delete the duplicate records in our original delta table. To do this we need to convert the `duplicate_comments` temp table to a pyarrow table that delta can utilize

In [24]:
duplicate_comments_pa_table = conn.sql("SELECT * FROM duplicate_comments").to_arrow_table()
duplicate_comments_pa_table

pyarrow.Table
comment_id: string
n_duplicates: int64
----
comment_id: [["UgzUwGsxgcpfwQcdVrJ4AaABAg","Ugw0mWXyiCv_G4K1ch54AaABAg","UgxBz7EV3q2lYeNtbKt4AaABAg","UgyQu70_1xaIcwgIuAF4AaABAg","UgwCEQ5Acjz78Neh9UR4AaABAg",...,"UgwjDnmM3Cbg_76k8Mt4AaABAg","UgwRf9S6cxH5vPevw8l4AaABAg","UgxSMQ6Uy69Jv_v0HMV4AaABAg","UgwFSTm4P3RCdA1JPgl4AaABAg","Ugxb58411DSukVjNkW94AaABAg"]]
n_duplicates: [[12500,12500,12500,12500,12500,...,3,3,3,3,3]]

In [26]:
delta_videos_comments_table.merge(
    duplicate_comments_pa_table,
    predicate="source.comment_id = target.comment_id",
    source_alias="source",
    target_alias="target",
    merge_schema=True
).when_matched_delete(
    
).execute()

# this transaction is equivalent to
# ```
# MERGE INTO delta_videos_comments_table AS source
# USING duplicate_comments_pa_table AS target
# ON source.comment_id = target.comment_id
# WHEN MATCHED THEN
#     DELETE;
# ```

{'num_source_rows': 4944,
 'num_target_rows_inserted': 0,
 'num_target_rows_updated': 0,
 'num_target_rows_deleted': 4086841,
 'num_target_rows_copied': 417,
 'num_output_rows': 417,
 'num_target_files_scanned': 2,
 'num_target_files_skipped_during_scan': 0,
 'num_target_files_added': 1,
 'num_target_files_removed': 1,
 'execution_time_ms': 9358,
 'scan_time_ms': 9192,
 'rewrite_time_ms': 5}

In [27]:
videos_comments_pa_table_new = delta_videos_comments_table.to_pyarrow_table()

In [28]:
conn.sql("""
    SELECT * FROM videos_comments_pa_table_new
""")

┌─────────┬─────────────┬───────────────────────────────────────────────────┬──────────────────────────┬───────────────────────────────────┬────────────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────┬──────────────────────┬────────────┬───────────────────────────────┬──────────────────────────────────────────────────────┬────────────────────────────┐
│  level  │  video_id   │                    comment_id                     │    author_channel_id     │ channel_id_where_comment_was_made │     parent_comment_id      │                                                     text_original                                                     │                                                   

In [30]:
conn.sql("""
    SELECT
        comment_id,
        COUNT(comment_id) AS n_duplicates
    FROM videos_comments_pa_table_new
    GROUP BY comment_id
    HAVING COUNT(comment_id) > 1
    ORDER BY n_duplicates DESC;
""")

┌────────────┬──────────────┐
│ comment_id │ n_duplicates │
│  varchar   │    int64     │
├────────────┴──────────────┤
│          0 rows           │
└───────────────────────────┘

# Once we finish the delete transaction and check if there are still duplicate comments, right after we will now get to the second stage of our transaction
which is reinserting the most recent version/record of those removed duplicate records we have saved as a temporary table earlier and which we now need to convert to a pyarrow table

In [25]:
most_recent_dups_pa_table = conn.sql("SELECT * FROM most_recent_dups").to_arrow_table()
most_recent_dups_pa_table

pyarrow.Table
level: string
video_id: string
comment_id: string
author_channel_id: string
channel_id_where_comment_was_made: string
parent_comment_id: string
text_original: string
text_display: string
published_at: string
updated_at: string
like_count: int64
author_display_name: string
author_channel_url: string
added_at: timestamp[us]
----
level: [["comment","comment","comment","comment","comment",...,"comment","comment","comment","comment","comment"]]
video_id: [["GE4S_68R0cg","BSlfQWpCFco","iqxq16AzJDw","5iA6bYz5FjM","TbMEMCvFbZk",...,"ybNrUQ1JMGE","8y-ViyUk7Dk","xorpL4rr2x8","3JTVQTk36R8","cWppAbqm9I8"]]
comment_id: [["UgzX7Yjm-1Bqy8-9YLF4AaABAg","UgzXFfG4kqcPhbjd2kd4AaABAg","UgzYW00hdncYAZxKA7Z4AaABAg","UgzZHR3es74IAP8BIzB4AaABAg","UgzaQtngvLMqtTmJQct4AaABAg",...,"UgwrJJWuwc6UBJKNYj14AaABAg","UgwsoNN3xDyc5S9uBmN4AaABAg","UgwtiCHTLsNy8l2hDwB4AaABAg","UgwwGd26Z066bxQ_QNp4AaABAg","UgwyOrP6EUcQPZXdn2t4AaABAg"]]
author_channel_id: [["UCULXWZalE7KRT4-UESAHDpg","UCPxLlUJSyFiJO88b3c2Np-g"

In [31]:
delta_videos_comments_table.merge(
    most_recent_dups_pa_table,
    predicate="target.comment_id = source.comment_id",
    source_alias="source",
    target_alias="target",
    merge_schema=True
).when_matched_update(
    updates={
        # these are not included as these are the composite keys
        # that are not by good practice supposed to be updated
        "level" : "source.level",
        # "video_id" : "source.video_id",
        # "comment_id" : "source.comment_id",
        "author_channel_id" : "source.author_channel_id",
        "channel_id_where_comment_was_made" : "source.channel_id_where_comment_was_made",
        "parent_comment_id" : "source.parent_comment_id",
        "text_original" : "source.text_original",
        "text_display" : "source.text_display",
        # "published_at" : "source.published_at",
        "updated_at" : "source.updated_at",
        "like_count" : "source.like_count",
        "author_display_name" : "source.author_display_name",
        "author_channel_url" : "source.author_channel_url",

        "added_at": "source.added_at", 
    }, 
    # this tells delta to only update a record if the new record
    # does indeed have changed its column values when compared to the
    # current record
    predicate="""
        (source.level IS DISTINCT FROM target.level) OR    
        (source.author_channel_id IS DISTINCT FROM target.author_channel_id) OR
        (source.channel_id_where_comment_was_made IS DISTINCT FROM target.channel_id_where_comment_was_made) OR
        (source.parent_comment_id IS DISTINCT FROM target.parent_comment_id) OR
        (source.text_original IS DISTINCT FROM target.text_original) OR
        (source.text_display IS DISTINCT FROM target.text_display) OR
        (source.updated_at > target.updated_at) OR
        (source.like_count IS DISTINCT FROM target.like_count) OR
        (source.author_display_name IS DISTINCT FROM target.author_display_name) OR
        (source.author_channel_url IS DISTINCT FROM target.author_channel_url) OR
        
        (source.added_at IS DISTINCT FROM target.added_at)
    """
    
    # (source.video_id IS DISTINCT FROM target.video_id) OR
    # (source.comment_id IS DISTINCT FROM target.comment_id) OR
    # (source.published_at IS DISTINCT FROM target.published_at) OR
).when_not_matched_insert_all()\
.execute()

{'num_source_rows': 4944,
 'num_target_rows_inserted': 4944,
 'num_target_rows_updated': 0,
 'num_target_rows_deleted': 0,
 'num_target_rows_copied': 0,
 'num_output_rows': 4944,
 'num_target_files_scanned': 2,
 'num_target_files_skipped_during_scan': 0,
 'num_target_files_added': 1,
 'num_target_files_removed': 0,
 'execution_time_ms': 3707,
 'scan_time_ms': 3687,
 'rewrite_time_ms': 3}

In [32]:
videos_comments_pa_table_new = delta_videos_comments_table.to_pyarrow_table()

In [33]:
conn.sql("SELECT * FROM videos_comments_pa_table_new")

┌─────────┬─────────────┬────────────────────────────┬──────────────────────────┬───────────────────────────────────┬───────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────┬──────────────────────┬────────────┬─────────────────────────┬─────────────────────────────────────────────────────┬────────────────────────────┐
│  level  │  video_id   │         comment_id         │    author_channel_id     │ channel_id_where_comment_was_made │ parent_comment_id │                                                     text_original                                                     │                                                                                           text_display                   

In [40]:
conn.sql("""
    SELECT
        comment_id,
        COUNT(comment_id) AS n_duplicates
    FROM videos_comments_pa_table_new
    GROUP BY comment_id
    HAVING COUNT(comment_id) > 1
    ORDER BY n_duplicates DESC;
""")

┌────────────┬──────────────┐
│ comment_id │ n_duplicates │
│  varchar   │    int64     │
├────────────┴──────────────┤
│          0 rows           │
└───────────────────────────┘

In [43]:
conn.sql("""
    SELECT
        COUNT(*) AS n_duplicates
    FROM videos_comments_pa_table_new
    GROUP BY ALL
    HAVING COUNT(*) > 1;
""")

┌──────────────┐
│ n_duplicates │
│    int64     │
├──────────────┤
│         5362 │
└──────────────┘

# Reddit posts

In [14]:
DELTA_POSTS_PATH = "../data/raw_reddit_posts"
delta_posts_table = DeltaTable(DELTA_POSTS_PATH)

df1 = delta_posts_table.to_pandas()

In [15]:
df1

,post_title,post_score,post_id,post_url,post_name,post_author_name,post_body,post_created_at,post_edited_at,added_at
0,Kpop Demon Hunters is nominated for Best Anima...,608,1p6gyco,https://i.redd.it/4zyoezehjf3g1.jpeg,t3_1p6gyco,Maximum-Ask-1745,,2025-11-25 16:28:02,1970-01-01 00:00:00,2025-11-26 08:38:45.407172
1,RAH THEY LOOK PERFECT,3104,1or05al,https://i.redd.it/rtlgsauzavzf1.jpeg,t3_1or05al,TLOU_1,,2025-11-08 01:11:53,1970-01-01 08:00:00,2025-11-09 20:33:25.623074
2,Ami Thompson appreciation post,1820,1os3fiy,https://www.reddit.com/gallery/1os3fiy,t3_1os3fiy,Trillex_PL_Zykov,There won't be enough of a praise for all the ...,2025-11-09 07:10:17,1970-01-01 08:00:00,2025-11-09 20:33:25.634669
3,Omg look at her 🥺,715,1os8xri,https://i.redd.it/6vkapl1ki50g1.jpeg,t3_1os8xri,Drawingandstuff2000,,2025-11-09 11:32:59,1970-01-01 08:00:00,2025-11-09 20:33:25.634669
4,I drew Mira! ✨,579,1os7kjs,https://www.reddit.com/gallery/1os7kjs,t3_1os7kjs,MayutArts,,2025-11-09 10:24:06,1970-01-01 08:00:00,2025-11-09 20:33:25.639084
5,K-Pop Demon Hunters Nominated for 5 Grammys,1247,1oqywhk,https://www.reddit.com/r/KpopDemonhunters/comm...,t3_1oqywhk,StrategicCarry,* Song of the Year for Golden (EJAE and Mark S...,2025-11-08 00:25:16,2025-11-08 00:27:11,2025-11-09 20:26:56.682074


In [18]:
test_posts_table = pa.table({
    "post_title":["new title", "new title too", "another new title"],
    "post_score": [9999, 10000, 24548],
    "post_id": ["1p6gyco", "mira", "zoey"],
    "post_url": ["https://i.redd.it/a0ecmeswtizf1.jpeg", "https://some-url.com", "https://some-url.com"],
    "post_name": ["t3_1p6gyco", "t3_mira", "t3_zoey"],
    "post_author_name": ["DemiFiendRSA", "Aristodemus", "leonidas"],
    "post_author_fullname": ["t2_DemiFiendRSA", "t2_Aristodemus", "t2_leonidas"],
    "post_body": ["test1", "test2", "test3"],
    "post_created_at": [dt.datetime.strptime("2025-11-06 03:44:31", "%Y-%m-%d %H:%M:%S"), dt.datetime.now(), dt.datetime.now()],
    "post_edited_at": [dt.datetime.now(), dt.datetime.strptime("1970-01-01 08:00:00", "%Y-%m-%d %H:%M:%S"), dt.datetime.strptime("1970-01-01 08:00:00", "%Y-%m-%d %H:%M:%S")],
    "added_at": [dt.datetime.now(), dt.datetime.now(), dt.datetime.now()]
})
test_posts_table

pyarrow.Table
post_title: string
post_score: int64
post_id: string
post_url: string
post_name: string
post_author_name: string
post_author_fullname: string
post_body: string
post_created_at: timestamp[us]
post_edited_at: timestamp[us]
added_at: timestamp[us]
----
post_title: [["new title","new title too","another new title"]]
post_score: [[9999,10000,24548]]
post_id: [["1p6gyco","mira","zoey"]]
post_url: [["https://i.redd.it/a0ecmeswtizf1.jpeg","https://some-url.com","https://some-url.com"]]
post_name: [["t3_1p6gyco","t3_mira","t3_zoey"]]
post_author_name: [["DemiFiendRSA","Aristodemus","leonidas"]]
post_author_fullname: [["t2_DemiFiendRSA","t2_Aristodemus","t2_leonidas"]]
post_body: [["test1","test2","test3"]]
post_created_at: [[2025-11-06 03:44:31.000000,2025-11-26 17:26:40.330449,2025-11-26 17:26:40.330449]]
post_edited_at: [[2025-11-26 17:26:40.330449,1970-01-01 08:00:00.000000,1970-01-01 08:00:00.000000]]
...

In [23]:
delta_posts_table.merge(
        test_posts_table,
        predicate="target.post_id = source.post_id",
        source_alias="source",
        target_alias="target",
        merge_schema=True
    ).when_matched_update(
        updates={
            # these are not included as these are the composite keys
            # that are not by good practice supposed to be updated
            # "post_id": "source.post_id",
            "post_title": "source.post_title",
            "post_score": "source.post_score",
            "post_url": "source.post_url",
            "post_name": "source.post_name",
            "post_author_name": "source.post_author_name",
            "post_author_fullname": "source.post_author_fullname",
            "post_body": "source.post_body",
            "post_created_at": "source.post_created_at",
            "post_edited_at": "source.post_edited_at",
            "added_at": "source.added_at",
        }, 
        # this tells delta to only update a record if the new record
        # does indeed have changed its column values when compared to the
        # current record
        predicate="source.post_title IS DISTINCT FROM target.post_title OR" \
            "source.post_score IS DISTINCT FROM target.post_score OR" \
            "source.post_url IS DISTINCT FROM target.post_url OR" \
            "source.post_name IS DISTINCT FROM target.post_name OR" \
            "source.post_author_name IS DISTINCT FROM target.post_author_name OR" \
            "source.post_author_fullname IS DISTINCT FROM target.post_author_fullname" \
            "source.post_body IS DISTINCT FROM target.post_body OR" \
            # "source.post_created_at IS DISTINCT FROM target.post_created_at OR" \
            "source.post_edited_at > target.post_edited_at OR" \
            "source.added_at IS DISTINCT FROM target.added_at OR"
    ).when_not_matched_insert(
        updates={
            "post_id": "source.post_id",
            "post_title": "source.post_title",
            "post_score": "source.post_score",
            "post_url": "source.post_url",
            "post_name": "source.post_name",
            "post_author_name": "source.post_author_name",
            "post_author_fullname": "source.post_author_fullname",
            "post_body": "source.post_body",
            "post_created_at": "source.post_created_at",
            "post_edited_at": "source.post_edited_at",
            "added_at": "source.added_at",
        }
    )\
    .execute()

Exception: External error: Schema error: Duplicate field name: post_author_fullname

In [19]:
DELTA_COMMENTS_PATH = "../data/raw_reddit_posts_comments"
delta_comments_table = DeltaTable(DELTA_COMMENTS_PATH)

df = delta_comments_table.to_pandas()

In [20]:
df.head()

,post_title,post_score,post_id,post_url,post_name,post_author_name,level,comment_id,comment_name,comment_upvotes,comment_downvotes,comment_created_at,comment_edited_at,comment_author_name,comment_author_fullname,comment_parent_id,comment_body
0,None,NaN,1oqywhk,None,t3_1oqywhk,None,comment,nnmd63c,t1_nnmd63c,414,0,2025-11-08 00:38:48,1970-01-01 08:00:00,Wannabe_Pear,t2_1rcscd9n8f,t3_1oqywhk,The fact everyone thought this was gonna be a ...
1,None,NaN,1oqywhk,None,t3_1oqywhk,None,reply,nnmjit2,t1_nnmjit2,76,0,2025-11-08 01:09:51,1970-01-01 08:00:00,SkullMan140,t2_10px8y,t1_nnmd63c,Who would have thought the story of the golden...
2,None,NaN,1oqywhk,None,t3_1oqywhk,None,reply,nnmwj7m,t1_nnmwj7m,40,0,2025-11-08 02:14:24,1970-01-01 08:00:00,Zimpiest_of_them_all,t2_1qnyyqmguk,t1_nnmd63c,I always love to imagine the faces of the peop...
3,None,NaN,1oqywhk,None,t3_1oqywhk,None,reply,nnmjoc7,t1_nnmjoc7,28,0,2025-11-08 01:10:37,1970-01-01 08:00:00,Enough-Ad-3111,t2_4r8syd06,t1_nnmd63c,Not to mention it has some potential for even ...
4,None,NaN,1oqywhk,None,t3_1oqywhk,None,reply,nnn2wsq,t1_nnn2wsq,13,0,2025-11-08 02:45:48,1970-01-01 08:00:00,Motor-Source8711,t2_x4md3w9pn,t1_nnmd63c,"when you add in total views of covers, views o..."


In [2]:
df.tail()

,post_title,post_score,post_id,post_url,post_name,post_author_name,level,comment_id,comment_name,comment_upvotes,comment_downvotes,comment_created_at,comment_edited_at,comment_author_name,comment_author_fullname,comment_parent_id,comment_body
37,Huntrix x Powerpuff girls,494,1opikw3,https://i.redd.it/a0ecmeswtizf1.jpeg,t3_1opikw3,Necessary_Board_9775,comment,nnbzzgf,t1_nnbzzgf,30,0,2025-11-06 08:04:19,1970-01-01 08:00:00,Nerdy-Everyday,t2_buc5n1jm,t3_1opikw3,Their personalities even line up.\n\nRumi / Bl...
38,EJAE and Mark Sonnenblick reveal an early iter...,154,1opqnxz,https://v.redd.it/1txc1qfcpkzf1,t3_1opqnxz,escarzador,comment,nnditt4,t1_nnditt4,32,0,2025-11-06 14:03:06,1970-01-01 08:00:00,mid-lev,t2_lfznrwzse,t3_1opqnxz,That woman in the background looks like she’s ...
39,EJAE and Mark Sonnenblick reveal an early iter...,154,1opqnxz,https://v.redd.it/1txc1qfcpkzf1,t3_1opqnxz,escarzador,reply,nndlsjn,t1_nndlsjn,12,0,2025-11-06 14:29:00,1970-01-01 08:00:00,PinkJenni,t2_1zdxy8rrbv,t1_nnditt4,She is taking it all in. EJAE’s voice is so he...
40,Rumi and EJAE are so similar!,319,1oplbfw,https://www.reddit.com/gallery/1oplbfw,t3_1oplbfw,Medical_Dimension919,comment,nncldfv,t1_nncldfv,39,0,2025-11-06 10:10:16,1970-01-01 08:00:00,Monkejedi,t2_surt50njj,t3_1oplbfw,Don’t forget Remi Ami!\n\nhttps://preview.redd...
41,Rumi and EJAE are so similar!,319,1oplbfw,https://www.reddit.com/gallery/1oplbfw,t3_1oplbfw,Medical_Dimension919,reply,nncm6cz,t1_nncm6cz,7,0,2025-11-06 10:15:05,1970-01-01 08:00:00,Medical_Dimension919,t2_8f9ixwa4q,t1_nncldfv,Of course! can't forget our queen!


https://delta-docs-incubator.netlify.app/delta-update/#slowly-changing-data-scd-type-2-operation-into-delta-tables

Upsert into a table using merge
You can upsert data from a source table, view, or DataFrame into a target Delta table by using the MERGE SQL operation. Delta Lake supports inserts, updates and deletes in MERGE, and it supports extended syntax beyond the SQL standards to facilitate advanced use cases.

Suppose you have a source table named people10mupdates or a source path at /tmp/delta/people-10m-updates that contains new data for a target table named people10m or a target path at /tmp/delta/people-10m. Some of these new records may already be present in the target data. To merge the new data, you want to update rows where the person’s id is already present and insert the new rows where no matching id is present. You can run the following:

```
MERGE INTO people10m
USING people10mupdates
ON people10m.id = people10mupdates.id
WHEN MATCHED THEN
  UPDATE SET
    id = people10mupdates.id,
    firstName = people10mupdates.firstName,
    middleName = people10mupdates.middleName,
    lastName = people10mupdates.lastName,
    gender = people10mupdates.gender,
    birthDate = people10mupdates.birthDate,
    ssn = people10mupdates.ssn,
    salary = people10mupdates.salary
WHEN NOT MATCHED
  THEN INSERT (
    id,
    firstName,
    middleName,
    lastName,
    gender,
    birthDate,
    ssn,
    salary
  )
  VALUES (
    people10mupdates.id,
    people10mupdates.firstName,
    people10mupdates.middleName,
    people10mupdates.lastName,
    people10mupdates.gender,
    people10mupdates.birthDate,
    people10mupdates.ssn,
    people10mupdates.salary
  )
  ```

# Lets test making a dummy updated record and upsert it into the existing delta lake table  

In [ ]:
created_at = dt.datetime.strptime("2025-11-06 08:04:19", "%Y-%m-%d %H:%M:%S")

datetime.datetime(2025, 11, 6, 8, 4, 19)

In [4]:
dt.datetime.now()

datetime.datetime(2025, 11, 8, 9, 18, 15, 317213)

In [19]:
test_table = pa.table({
    "post_title": ["new title", "new title too", "another new title"],
    "post_score": [9999, 10000, 24548],
    "post_id": ["1opcev8", "mira", "zoey"],
    "post_url": ["https://i.redd.it/a0ecmeswtizf1.jpeg", "https://some-url.com", "https://some-url.com"],
    "post_name": ["t3_1opcev8", "t3_mira", "t3_zoey"],
    "post_author_name": ["DemiFiendRSA", "Aristodemus", "leonidas"],
    "level": ["comment", "comment", "reply"],
    "comment_upvotes": [1000, 2000, 3000],
    "comment_downvotes": [6, 0, 0],
    "comment_name": ["t1_nnakzjc", "t1_someguy", "t1_anotherguy"],
    "comment_id": ["nnakzjc", "someguy", "anotherguy"],
    "comment_created_at": [dt.datetime.strptime("2025-11-06 03:44:31", "%Y-%m-%d %H:%M:%S"), dt.datetime.now(), dt.datetime.now()],
    "comment_edited_at": [dt.datetime.now(), dt.datetime.strptime("1970-01-01 08:00:00", "%Y-%m-%d %H:%M:%S"), dt.datetime.strptime("1970-01-01 08:00:00", "%Y-%m-%d %H:%M:%S")],
    "comment_author_name": ["Cluelessbigirl", "SomeGuy123", "anotherguy123"],
    "comment_author_fullname": ["t2_duy9z7kw", "t2_4234982398", "t2_0feie23"],
    "comment_parent_id": ["t3_1opcev8", "t3_mira", "t3_zoey"],
    "comment_body": ["We're going up up up, it's our moment!", "fit check for my napalm eraaa", "need to beat my face make it cute and savage"],
})
test_table

pyarrow.Table
post_title: string
post_score: int64
post_id: string
post_url: string
post_name: string
post_author_name: string
level: string
comment_upvotes: int64
comment_downvotes: int64
comment_name: string
comment_id: string
comment_created_at: timestamp[us]
comment_edited_at: timestamp[us]
comment_author_name: string
comment_author_fullname: string
comment_parent_id: string
comment_body: string
----
post_title: [["new title","new title too","another new title"]]
post_score: [[9999,10000,24548]]
post_id: [["1opcev8","mira","zoey"]]
post_url: [["https://i.redd.it/a0ecmeswtizf1.jpeg","https://some-url.com","https://some-url.com"]]
post_name: [["t3_1opcev8","t3_mira","t3_zoey"]]
post_author_name: [["DemiFiendRSA","Aristodemus","leonidas"]]
level: [["comment","comment","reply"]]
comment_upvotes: [[1000,2000,3000]]
comment_downvotes: [[6,0,0]]
comment_name: [["t1_nnakzjc","t1_someguy","t1_anotherguy"]]
...

# now we merge the updated records and newly inserted records into the existing delta lake table based on the unique ids such as post_name/post_id, comment_name/comment_id, and comment_parent_id as these composite keys make the records unique and prevent duplication

what if a target delta lake table has a composite key that exists but has been deleted when scraped, I guess in this case as we don't need to already since we are doing this on a daily basis, and we only want to get the information left by users in a forum purely, doesn't matter if their accounts get deleted, we don't need to reflect this change in our data lake house table



which is basically the equivalent of
```
MERGE INTO delta_table
USING test_table
ON delta_table.post_id = source_table.post_id AND
target.comment_id = source.comment_id AND 
target.comment_parent_id = source.comment_parent_id
WHEN MATCHED AND (
    -- The Conditional Update Predicate (Your Change Check)
    source.post_title IS DISTINCT FROM target.post_title OR
    source.post_score IS DISTINCT FROM target.post_score OR
    source.post_url IS DISTINCT FROM target.post_url OR
    source.post_name IS DISTINCT FROM target.post_name OR
    source.post_author_name IS DISTINCT FROM target.post_author_name OR
    source.level IS DISTINCT FROM target.level OR
    source.comment_upvotes IS DISTINCT FROM target.comment_upvotes OR
    source.comment_downvotes IS DISTINCT FROM target.comment_downvotes OR
    source.comment_name IS DISTINCT FROM target.comment_name OR
    source.comment_edited_at > target.comment_edited_at OR 
    source.comment_author_name IS DISTINCT FROM target.comment_author_name OR
    source.comment_author_fullname IS DISTINCT FROM target.comment_author_fullname OR
    source.comment_body IS DISTINCT FROM target.comment_body
) THEN
    UPDATE SET
        target.post_title = source.post_title,
        target.post_score = source.post_score,
        target.post_url = source.post_url,
        target.post_name = source.post_name,
        target.post_author_name = source.post_author_name,
        target.level = source.level,
        target.comment_upvotes = source.comment_upvotes,
        target.comment_downvotes = source.comment_downvotes,
        target.comment_name = source.comment_name,
        target.comment_edited_at = source.comment_edited_at, -- Must update the column that triggers the change!
        target.comment_author_name = source.comment_author_name,
        target.comment_author_fullname = source.comment_author_fullname,
        target.comment_body = source.comment_body
WHEN NOT MATCHED THEN 
    INSERT (
        post_id
        comment_id
        comment_parent_id
        post_title
        post_score
        post_url
        post_name
        post_author_name
        level
        comment_upvotes
        comment_downvotes
        comment_name
        comment_created_at
        comment_edited_at
        comment_author_name
        comment_author_fullname
        comment_body
    )
    VALUES (
        source.post_id
        source.comment_id
        source.comment_parent_id
        source.post_title
        source.post_score
        source.post_url
        source.post_name
        source.post_author_name
        source.level
        source.comment_upvotes
        source.comment_downvotes
        source.comment_name
        source.comment_created_at
        source.comment_edited_at
        source.comment_author_name
        source.comment_author_fullname
        source.comment_body
    )
```

In [20]:
delta_table.merge(
    test_table,
    predicate="target.post_id = source.post_id AND \
        target.comment_id = source.comment_id AND \
        target.comment_parent_id = source.comment_parent_id",
    source_alias="source",
    target_alias="target"
).when_matched_update(
    updates={
        # these are not included as these are the composite keys
        # that are not by good practice supposed to be updated
        # "post_id": "source.post_id",
        # "comment_id": "source.comment_id",
        # "comment_parent_id": "source.comment_parent_id",
        "post_title": "source.post_title",
        "post_score": "source.post_score",
        "post_url": "source.post_url",
        "post_name": "source.post_name",
        "post_author_name": "source.post_author_name",
        "level": "source.level",
        "comment_upvotes": "source.comment_upvotes",
        "comment_downvotes": "source.comment_downvotes",
        "comment_name": "source.comment_name",
        # "comment_created_at": "source.comment_created_at",
        "comment_edited_at": "source.comment_edited_at",
        "comment_author_name": "source.comment_author_name",
        "comment_author_fullname": "source.comment_author_fullname",
        "comment_body": "source.comment_body"
    }, 
    # this tells delta to only update a record if the new record
    # does indeed have changed its column values when compared to the
    # current record
    predicate="source.post_title IS DISTINCT FROM target.post_title OR" \
        "source.post_score IS DISTINCT FROM target.post_score OR" \
        "source.post_url IS DISTINCT FROM target.post_url OR" \
        "source.post_name IS DISTINCT FROM target.post_name OR" \
        "source.post_author_name IS DISTINCT FROM target.post_author_name OR" \
        "source.level IS DISTINCT FROM target.level OR" \
        "source.comment_upvotes IS DISTINCT FROM target.comment_upvotes OR" \
        "source.comment_downvotes IS DISTINCT FROM target.comment_downvotes OR" \
        "source.comment_name IS DISTINCT FROM target.comment_name OR" \
        # "source.comment_created_at IS DISTINCT FROM target.comment_created_at OR" \
        "source.comment_edited_at > target.comment_edited_at OR" \
        "source.comment_author_name IS DISTINCT FROM target.comment_author_name OR" \
        "source.comment_author_fullname IS DISTINCT FROM target.comment_author_fullname OR" \
        "source.comment_body IS DISTINCT FROM target.comment_body" \
).when_not_matched_insert_all()\
.execute()

{'num_source_rows': 3,
 'num_target_rows_inserted': 1,
 'num_target_rows_updated': 0,
 'num_target_rows_deleted': 0,
 'num_target_rows_copied': 0,
 'num_output_rows': 1,
 'num_target_files_scanned': 1,
 'num_target_files_skipped_during_scan': 0,
 'num_target_files_added': 1,
 'num_target_files_removed': 0,
 'execution_time_ms': 82,
 'scan_time_ms': 40,
 'rewrite_time_ms': 0}

# after update and insertion of these 2 records we will check the delta table again

In [21]:
new_df = delta_table.to_pandas()
new_df.head()

,post_title,post_score,post_id,post_url,post_name,post_author_name,level,comment_id,comment_name,comment_upvotes,comment_downvotes,comment_created_at,comment_edited_at,comment_author_name,comment_author_fullname,comment_parent_id,comment_body
0,another new title,24548,zoey,https://some-url.com,t3_zoey,leonidas,reply,anotherguy,t1_anotherguy,3000,0,2025-11-08 13:13:10.800989,1970-01-01 08:00:00.000000,anotherguy123,t2_0feie23,t3_zoey,need to beat my face make it cute and savage
1,new title too,10000,mira,https://some-url.com,t3_mira,Aristodemus,comment,someguy,t1_someguy,2000,0,2025-11-08 09:29:41.414516,1970-01-01 08:00:00.000000,SomeGuy123,t2_4234982398,t3_mira,fit check for my napalm eraaa
2,new title,9999,1opcev8,https://i.redd.it/a0ecmeswtizf1.jpeg,t3_1opcev8,DemiFiendRSA,comment,nnakzjc,t1_nnakzjc,1000,6,2025-11-06 03:44:31.000000,2025-11-08 09:29:41.414516,Cluelessbigirl,t2_duy9z7kw,t3_1opcev8,"We're going up up up, it's our moment!"
3,‘KPop Demon Hunters 2’ Aims for 2029 Release o...,2121,1opcev8,https://variety.com/2025/film/news/kpop-demon-...,t3_1opcev8,DemiFiendRSA,reply,nnaold6,t1_nnaold6,205,0,2025-11-06 04:01:42.000000,1970-01-01 08:00:00.000000,Hero_of_the_toons,t2_64w7o0ao,t1_nnakzjc,Wouldn’t want a Kpdh equivalent of Sonic 06 if...
4,‘KPop Demon Hunters 2’ Aims for 2029 Release o...,2121,1opcev8,https://variety.com/2025/film/news/kpop-demon-...,t3_1opcev8,DemiFiendRSA,reply,nnb3wf6,t1_nnb3wf6,79,0,2025-11-06 05:15:41.000000,1970-01-01 08:00:00.000000,CelineShotFirst,t2_1rqf9pzis9,t1_nnakzjc,The original film took 7 years from pitch to r...
